In [ ]:
%matplotlib inline

In [ ]:
import os
os.environ["KERAS_BACKEND"] = "tensorflow"
import keras
import tensorflow as tf
import keras_hub
import numpy as np
import json
import sklearn
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
os.getcwd()

In [ ]:
# show GPU device(s):
print(tf.config.list_physical_devices('GPU'))

In [ ]:
# datasets and model directories
cache_dir = os.environ['HOME']
imdb_path = 'imdb.npz'
imdb_word_index_path = os.path.join(cache_dir, '.keras', 'datasets', 'imdb_word_index.json')
model_path = os.path.join(cache_dir, '.kagglehub/models/keras/bert/keras/bert_medium_en_uncased/3')

In [ ]:
# "best" trained weights directory
weights_dir = 'weights'
os.makedirs(weights_dir, exist_ok=True)

In [ ]:
# Load IMDB dataset, consisting of 25,000 movie reviews from IMDB, 
# labeled by sentiment (positive/negative)
# get features and labels
# Use the default parameters to keras.datasets.imdb.load_data
start_char = 1
oov_char = 2
index_from = 3

(x_train, y_train), (x_test, y_test) = keras.datasets.imdb.load_data(
    path=imdb_path,
    num_words=None,
    skip_top=0,
    maxlen=None,
    seed=113,
    start_char=start_char,
    oov_char=oov_char,
    index_from=index_from,
)

# split test into validation and test 50/50
x_val = x_test[:(x_test.shape[0]//2)]
y_val = y_test[:(y_test.shape[0]//2)]
x_test = x_test[(x_test.shape[0]//2):]
y_test = y_test[(y_test.shape[0]//2):]

In [ ]:
x_train

In [ ]:
y_train

In [ ]:
# trunkate and pad short sequences. oov_char is 2
maxlen = 512
x_train = keras.preprocessing.sequence.pad_sequences(x_train, maxlen=maxlen, 
                                                     padding='post', 
                                                     truncating='post', 
                                                     value=oov_char)
x_val = keras.preprocessing.sequence.pad_sequences(x_val, 
                                                   maxlen=maxlen, 
                                                   padding='post', 
                                                   truncating='post', 
                                                   value=oov_char)
x_test = keras.preprocessing.sequence.pad_sequences(x_test, 
                                                    maxlen=maxlen, 
                                                    padding='post', 
                                                    truncating='post', 
                                                    value=oov_char)

In [ ]:
# read dictionary with word index mapping
with open(imdb_word_index_path) as f:
    word_index = json.load(f)
word_index

In [ ]:
# Reverse the word index to obtain a dict mapping indices to words
# And add `index_from` to indices to sync with `x_train`
inverted_word_index = dict(
    (i + index_from, word) for (word, i) in word_index.items()
)
# Update `inverted_word_index` to include `start_char` and `oov_char`
inverted_word_index[start_char] = "[START]"
inverted_word_index[oov_char] = "[OOV]"

# decode the sequences:
x_train_decoded = [" ".join(inverted_word_index[i] for i in x) for x in x_train]
x_val_decoded = [" ".join(inverted_word_index[i] for i in x) for x in x_val]
x_test_decoded = [" ".join(inverted_word_index[i] for i in x) for x in x_test]

In [ ]:
x_train_decoded

In [ ]:
# Pretrained classifier.
classifier = keras_hub.models.BertClassifier.from_preset(
    model_path,
    num_classes=2,  # binary classification (positive/negative)
)

# Re-compile (e.g., with a new learning rate).
classifier.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=keras.optimizers.Adam(5e-6),  # 5e-5
    jit_compile=True,
)

# Access backbone programmatically (e.g., to change `trainable`).
classifier.backbone.trainable = True  # enable/disable fine-tuning.

In [ ]:
# show backbone summary
classifier.backbone.summary()

In [ ]:
# plot model summary
keras.utils.plot_model(classifier.backbone, show_shapes=True, 
                       show_layer_activations=True, 
                       show_trainable=True, dpi=30)

In [ ]:
# show classifier summary
classifier.summary()

In [ ]:
# plot model summary
keras.utils.plot_model(classifier, show_shapes=True, 
                       show_layer_activations=True, 
                       show_trainable=True, dpi=60)

In [ ]:
# Train the classifier, with early stopping and weights saving
# earlystop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)
earlystop = keras.callbacks.EarlyStopping(monitor='val_sparse_categorical_accuracy', mode='max', patience=3)
checkpoint = keras.callbacks.ModelCheckpoint(
    filepath=os.path.join(weights_dir, 'bert_base_en_classifier_imdb_finetuning.best.keras'),
    # monitor='val_loss',  # val_sparse_categorical_accuracy
    # mode='min',  # max
    monitor='val_sparse_categorical_accuracy',
    mode='max',
    save_best_only=True)

history = classifier.fit(x=x_train_decoded, y=y_train, 
                         batch_size=128, epochs=100, 
                         validation_data=(x_val_decoded, y_val), 
                         callbacks=[earlystop, checkpoint])

In [ ]:
# save the last update
classifier.save(os.path.join(weights_dir, 'bert_base_en_classifier_imdb_finetuning.last.keras'))

In [ ]:
# show training history (loss, accuracy)
fig, axes = plt.subplots(2, 1, sharex=True, figsize=(6, 6))
axes[0].semilogy(history.history['loss'])
axes[0].semilogy(history.history['val_loss'])
axes[0].set_ylabel('loss')
axes[0].legend(['train', 'val'], loc='upper right')

axes[1].plot(history.history['sparse_categorical_accuracy'])
axes[1].plot(history.history['val_sparse_categorical_accuracy'])
axes[1].set_ylabel('sparse_categorical_accuracy')
axes[1].set_xlabel('epoch')
axes[1].legend(['train', 'val'], loc='lower right')

In [ ]:
# optional; reload best/last model for predictions
classifier = keras.models.load_model(
    filepath=os.path.join(weights_dir, 
                          'bert_base_en_classifier_imdb_finetuning.best.keras'),
                        custom_objects=None, compile=False, safe_mode=True)

In [ ]:
# get predictions
pred_train = classifier.predict(x_train_decoded)
pred_val = classifier.predict(x_val_decoded)
pred_test = classifier.predict(x_test_decoded)

In [ ]:
# convert to labels (no need to convert logits to probabilities)
pred_train_labels = np.argmax(pred_train, axis=1)
pred_val_labels = np.argmax(pred_val, axis=1)
pred_test_labels = np.argmax(pred_test, axis=1)

In [ ]:
# predicted "probabilities" for positive class
pred_prob_train = keras.ops.softmax(pred_train)[:, 1]
pred_prob_val = keras.ops.softmax(pred_val)[:, 1]
pred_prob_test = keras.ops.softmax(pred_test)[:, 1]

In [ ]:
# plot ROC curves
fig, ax = plt.subplots(1, 1)
for y, y_prob, label in zip([y_train, y_val, y_test], 
                            [pred_prob_train, pred_prob_val, pred_prob_test], 
                            ['train', 'val', 'test']):
    fpr, tpr, _ = sklearn.metrics.roc_curve(y, y_prob)
    ax.plot(fpr, tpr, label=label)
ax.plot([0, 1], [0, 1], 'k:', label='_nolegend_')
plt.legend(loc='best')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')

In [ ]:
# precision recall curves
fig, ax = plt.subplots(1, 1)
for y, y_prob, label in zip([y_train, y_val, y_test], 
                            [pred_prob_train, pred_prob_val, pred_prob_test], 
                            ['train', 'val', 'test']):
    precision, recall, _ = sklearn.metrics.precision_recall_curve(y, y_prob)
    ax.plot(recall, precision, label=label)
plt.legend(loc='best')
plt.xlabel('Recall')
plt.ylabel('Precision')

In [ ]:
# print out some common metrics
metrics = pd.DataFrame(columns=['accuracy', 'precision', 'recall', 'f1', 
                                'ROCAUC', 'AP', 'bal_accuracy', 
                                'MCC', 'TP', 'FP', 'FN', 'TN'])
skm = sklearn.metrics

for y, pred, prob, label in zip([y_train, y_val, y_test], 
                          [pred_train_labels, pred_val_labels, pred_test_labels],
                          [pred_prob_train, pred_prob_val, pred_prob_test],
                          ['train', 'val', 'test']):
    metrics.loc[label, 'accuracy'] = skm.accuracy_score(y, pred)
    metrics.loc[label, 'precision'] = skm.precision_score(y, pred)
    metrics.loc[label, 'recall'] = skm.recall_score(y, pred)
    metrics.loc[label, 'f1'] = skm.f1_score(y, pred)
    metrics.loc[label, 'ROCAUC'] = skm.roc_auc_score(y, prob)
    metrics.loc[label, 'AP'] = skm.average_precision_score(y, prob)
    metrics.loc[label, 'bal_accuracy'] = skm.balanced_accuracy_score(y, pred)
    metrics.loc[label, 'MCC'] = skm.matthews_corrcoef(y, pred)
    (metrics.loc[label, 'TP'],
     metrics.loc[label, 'FP'],
     metrics.loc[label, 'FN'],
     metrics.loc[label, 'TN']) = skm.confusion_matrix(y, pred).ravel()

metrics